# Feature Engineering 4
***
我们这回使用一个更加精巧的方法吧

In [1]:
%load_ext autoreload
%autoreload 2

import gc
import sys
import time
import pandas as pd
import numpy as np
from datetime import datetime
from scipy.stats import mode
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

sys.path.append('/root/code/elo/Tools/')
from tools import *

In [2]:
new = pd.read_csv('/root/data/new_merchant_transactions.csv')
his = pd.read_csv('/root/data/historical_transactions.csv')
train = pd.read_csv('/root/data/train.csv')
test = pd.read_csv('/root/data/test.csv')

In [3]:
#对 his 与 new 的时间数据进行处理，创建特征该次交易距离起始时间的days
his['purchase_date'] = pd.to_datetime(his.purchase_date)
new['purchase_date'] = pd.to_datetime(new.purchase_date)
startime = his.purchase_date.min()

for df in [his, new]:
    df['days_to_start'] = (df.purchase_date - startime).dt.days
    df['month_diff'] = ((datetime.today() - df['purchase_date']).dt.days)//30
    df['month_diff'] += df['month_lag']
    df['dayofweek'] = df.purchase_date.apply(lambda t: t.dayofweek)
    df['month'] = df.purchase_date.apply(lambda t: t.month)
    df['weekend'] = (df.purchase_date.dt.weekday >=5).astype(int)
    df['hour'] = df['purchase_date'].dt.hour


In [4]:
for df in [his, new]:
    df['category_2'].fillna(1.0,inplace=True)
    df['category_3'].fillna('A',inplace=True)
    df['merchant_id'].fillna('M_ID_00a6ca8a8a',inplace=True)

In [5]:
# authorized_flag
new['authorized_flag'] = new.authorized_flag.map({'N':0,'Y':1})
his['authorized_flag'] = his.authorized_flag.map({'N':0,'Y':1})

agg = {
    'authorized_flag':['mean', 'sum']
}
auth = his.groupby('card_id').agg(agg)
auth.columns = ['_'.join(col).strip() for col in auth.columns.values]
auth.reset_index(inplace=True)


In [6]:
agg = {'authorized_flag': ['mean', 'count']}
flag_month = his.groupby(['card_id', 'month_lag']).agg(agg)
flag_month.columns = ['_'.join(col).strip() for col in flag_month.columns.values]
flag_month.reset_index(inplace=True)

# 找到对于每id来说，距离评估日最后的月成交数量与成交率
col_idx = ['authorized_flag_mean', 'authorized_flag_count']
row_idx = flag_month.groupby('card_id').month_lag.apply(lambda x: x.idxmax()).values
final = flag_month.loc[row_idx,col_idx].reset_index(drop=True)
final.columns = ['final_authorized_flag_mean', 'final_authorized_flag_count']


# 找到对于每id来说，最开始的月成交数量与成交率
col_idx = ['authorized_flag_mean', 'authorized_flag_count']
row_idx = flag_month.groupby('card_id').month_lag.apply(lambda x: x.idxmin()).values
start = flag_month.loc[row_idx,col_idx].reset_index(drop=True)
start.columns = ['start_authorized_flag_mean', 'start_authorized_flag_count']


auth['final_start_authorized_flag_count_percent'] = final.final_authorized_flag_count / start.start_authorized_flag_count
auth['final_start_start_authorized_flag_mean_minues'] = final.final_authorized_flag_mean - start.start_authorized_flag_mean
del col_idx, row_idx, flag_month, final, start; gc.collect()

78

In [7]:
his.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,days_to_start,month_diff,dayofweek,month,weekend,hour
0,1,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37,175,10,6,6,1,15
1,1,C_ID_4e6213e9bc,88,N,0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16,195,10,5,7,1,12
2,1,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37,220,11,2,8,0,22
3,1,C_ID_4e6213e9bc,88,N,0,A,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,1.0,16,34,244,11,5,9,1,10
4,1,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,1.0,16,37,68,11,4,3,0,1


In [8]:
def agg_dataframe(df):
    
    df['category_1'] = df.category_1.map({'N':0, 'Y':1})
    df['log_amount'] = np.log(df.purchase_amount+0.75)
    df = pd.get_dummies(df, columns=['category_2', 'category_3'])
    
    agg = {
        'city_id': ['nunique'],
        'merchant_category_id': ['nunique'],
        'merchant_id': ['nunique'],
        'state_id': ['nunique'],
        'subsector_id': ['nunique'],
        'installments': ['sum', 'mean', 'max', 'min', 'std'],
        'category_1': ['mean', 'sum'],
        'category_2_1.0': ['mean'],
        'category_2_2.0': ['mean'],
        'category_2_3.0': ['mean'],
        'category_2_4.0': ['mean'],
        'category_2_5.0': ['mean'],
        'category_3_A': ['mean'],
        'category_3_B': ['mean'],
        'category_3_C': ['mean'],
        'month_lag': ['min', 'max', 'mean', 'std'],
        'days_to_start': [np.ptp, 'min', 'max'],
        'purchase_amount': ['sum', 'mean', 'max', 'min', 'std'],
        'log_amount': ['sum', 'mean', 'max', 'min', 'std'],
        'month_diff': ['mean'],
        'dayofweek': ['nunique', 'std'],
        'weekend':['mean'],
        'hour':['nunique'],
        'month': ['std','nunique'],
        'card_id': ['size']
    }
    agg_df = df.groupby(['card_id']).agg(agg)
    agg_df.columns = ['_'.join(col).strip() for col in agg_df.columns.values]
    agg_df.reset_index(inplace=True)
    
    return agg_df

In [9]:
agg_his = agg_dataframe(his)
agg_his.columns = ['his_' + c if c != 'card_id' else c for c in agg_his.columns]
agg_new = agg_dataframe(new)
agg_new.columns = ['new_' + c if c != 'card_id' else c for c in agg_new.columns]

In [10]:
agg_his['agg_his_date_average'] = agg_his['his_days_to_start_ptp']/agg_his['his_card_id_size']
agg_new['agg_new_date_average'] = agg_new['new_days_to_start_ptp']/agg_new['new_card_id_size']

In [11]:
train = pd.merge(train, auth, on='card_id', how='left')
train = pd.merge(train, agg_his, on='card_id', how='left')
train = pd.merge(train, agg_new, on='card_id', how='left')

test = pd.merge(test, auth, on='card_id', how='left')
test = pd.merge(test, agg_his, on='card_id', how='left')
test = pd.merge(test, agg_new, on='card_id', how='left')

In [12]:
for df in [train, test]:
    df['first_active_month'] = (pd.to_datetime(df['first_active_month']) - startime).dt.days
    df['his_first_buy'] = (df['his_days_to_start_min'] - df['first_active_month'])
    df['new_first_buy'] = (df['new_days_to_start_min'] - df['first_active_month'])
    df['total_count'] = df['his_card_id_size']+df['new_card_id_size']

In [13]:
train['outliers'] = 0
train.loc[train['target'] < -30, 'outliers'] = 1
train['outliers'].value_counts()

ore = train.groupby('feature_1')['outliers'].mean()

In [14]:
for f in ['feature_1','feature_2','feature_3']:
    order_label = train.groupby([f])['outliers'].mean()
    train[f] = train[f].map(order_label)
    test[f] = test[f].map(order_label)

In [15]:
train.to_csv('/root/tempfile/train_final191219.csv',index=False)
test.to_csv('/root/tempfile/test_final191219.csv',index=False)